In [9]:
import pandas as pd
import numpy as np

import tensorflow as tf
import os

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [10]:
base_model = ResNet50(include_top=False,
                      weights=None,
                      input_shape=(224,224,3))

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001))(x)

model = Model(inputs=base_model.input, outputs=x)

learning_rate = 0.0001
model. compile(optimizer=Adam(learning_rate=learning_rate),
              loss='binary_crossentropy',  # sparse_categotical_crossentropy
              metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [5]:
# base path
base_dir = 'D:/archive/Humans/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
train_face_dir = os.path.join(train_dir, 'face')
validation_face_dir = os.path.join(validation_dir, 'face')

In [11]:
train_datagen = ImageDataGenerator(rescale= 1./255,
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale= 1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=32,
                                                    color_mode='rgb',
                                                    class_mode='binary', #categorical, sparse
                                                    target_size=(224,224))
validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              batch_size=32,
                                                              color_mode='rgb',
                                                              class_mode='binary', #categorical, sparse
                                                              target_size=(224,224))



Found 5463 images belonging to 1 classes.
Found 1756 images belonging to 1 classes.


In [12]:
train_generator.class_indices

{'face': 0}

In [13]:
model.fit(train_generator,
          validation_data = validation_generator,
          epochs=10)

Epoch 1/10
 11/171 [>.............................] - ETA: 16:52 - loss: 0.1441 - accuracy: 0.9574

c:\Users\Hong\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


171/171 [==============================] - 1214s 7s/step - loss: 0.0116 - accuracy: 0.9973 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 2/10
 41/171 [======>.......................] - ETA: 14:20 - loss: 0.0021 - accuracy: 1.0000

KeyboardInterrupt: 

In [ ]:
model.save('face_detection.h5')